<a href="https://colab.research.google.com/github/mohamed32145/MindCare/blob/main/filesArrangment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" we will authenticate colab to access to our data on google drive to start organize and
 prepare the images fodlers

 """
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Step 3: Authenticate and create PyDrive2 client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
"""
---

**Step 1: Data Organization**

The dataset was first organized into the following diagnostic groups:

* **CN** (Cognitively Normal)
* **sMCI** (Stable Mild Cognitive Impairment)
* **pMCI** (Progressive Mild Cognitive Impairment)
* **AD** (Alzheimer’s Disease)

---

 Image Selection Criteria

 For the AD and CN groups:

* We retained only the images with the following preprocessing pipeline:
  `MPR-R__GradWarp__B1_Correction__N3__Scaled` or
  `MPR__GradWarp__B1_Correction__N3__Scaled`.

* Images labeled with `*_scaled_2*` were removed.
  These images were adjusted to disable scaling on axes where the phantom's measurements are considered unreliable.
  While `*_scaled_2*` images may offer higher accuracy, they are significantly fewer in number and **cannot be mixed** with the standard `*_scaled*` images for model training due to consistency issues.

---

 For the sMCI and pMCI groups:

* Due to the limited availability of subjects in these groups, we included:

  * `MPR-R__GradWarp__B1_Correction__N3__Scaled`
  * `MPR__GradWarp__B1_Correction__N3__Scaled`
  * `MPR__GradWarp__N3__Scaled`

* Subjects with only `MPR__GradWarp__N3__Scaled` images will undergo additional preprocessing, specifically:

  *Bias field correction, to reduce intensity non-uniformity and improve image quality before training.







 """

#a reusable function that search and delete the scaled_2 file in the all groups (AD, CN, pMCI, sMCI)

def delete_scaled_2_folders_in_group(parent_folder_id, group_name):
    print(f"\ Processing group: {group_name}")

    # Get all subject folders inside the parent group folder (e.g., AD)
    subject_folders = drive.ListFile({
        'q': f"'{parent_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    }).GetList()

    for subject in subject_folders:
        print(f" Checking subject: {subject['title']}")

        # Get subfolders inside each subject folder
        subfolders = drive.ListFile({
            'q': f"'{subject['id']}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
        }).GetList()

        for folder in subfolders:
            if folder['title'].endswith('Scaled_2'):
                print(f" Deleting folder: {folder['title']} in subject {subject['title']}")
                folder.Delete()


AD_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
CN_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
PMCI_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
SMCI_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'

# Call the function for each group
delete_scaled_2_folders_in_group(AD_FOLDER_ID, 'AD')
delete_scaled_2_folders_in_group(CN_FOLDER_ID, 'CN')
delete_scaled_2_folders_in_group(PMCI_FOLDER_ID, 'pMCI')
delete_scaled_2_folders_in_group(SMCI_FOLDER_ID, 'sMCI')


In [ ]:

"""
now we consider the case where a subject has two kinds of scanning sessions one
(two fodlers) one starting with MPR and the other MPR-R .MPR probably stands for "Multiplanar Reconstruction,
" a technique that generates images in multiple planes (e.g., axial, sagittal, coronal) from a single acquisition.
MPR-R may indicate a refined or revised version of the MPR protocol, potentially with improved quality or specific
modifications for certain research purposes

MPR-R is less by far than MPR, some subjects has MPR-R images more than the MPR
images, just in this case we will choose the MPR-R images

"""


#we run this function on all the folders
def clean_mpr_vs_mprr(drive_folder_id):
    subject_folders = drive.ListFile({'q': f"'{drive_folder_id}' in parents and trashed=false"}).GetList()

    for subject in subject_folders:
        if subject['mimeType'] != 'application/vnd.google-apps.folder':
            continue

        subject_id = subject['id']
        subject_name = subject['title']
        print(f"\n Checking subject: {subject_name}")

        subfolders = drive.ListFile({'q': f"'{subject_id}' in parents and trashed=false"}).GetList()

        mpr_folder = None
        mprr_folder = None
        mpr_count = 0
        mprr_count = 0

        for f in subfolders:
            name = f['title']
            if f['mimeType'] != 'application/vnd.google-apps.folder':
                continue

            if name.startswith('MPR__') and not name.startswith('MPR-R__'):
                contents = drive.ListFile({'q': f"'{f['id']}' in parents and trashed=false"}).GetList()
                mpr_folder = f
                mpr_count = len(contents)

            elif name.startswith('MPR-R__'):
                contents = drive.ListFile({'q': f"'{f['id']}' in parents and trashed=false"}).GetList()
                mprr_folder = f
                mprr_count = len(contents)

        if mpr_folder and mprr_folder:
            print(f" MPR: {mpr_count} files, MPR-R: {mprr_count} files")

            # Prefer MPR-R if same or more files
            if mprr_count >= mpr_count:
                print(" Deleting MPR")
                drive.CreateFile({'id': mpr_folder['id']}).Trash()
            else:
                print(" Deleting MPR-R")
                drive.CreateFile({'id': mprr_folder['id']}).Trash()
        else:
            print(" One or both MPR versions not found.")



folder_ids = {
    'AD': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'PMCI': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'sMCI': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'CN': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU''
}

for name, folder_id in folder_ids.items():
    print(f"\n Processing: {name}")
    clean_mpr_vs_mprr(folder_id)



In [ ]:
"""
the above function keep files such MPR__GradWarp__N3__Scaled in all the folder
for the AN, CN  groups we dont need them for cause the images availbility
so we will delete them

"""

import os
import shutil

def clean_subjects_with_mpr(root_dirs):
    keep_folders = {
        "MPR__GradWarp__B1_Correction__N3__Scaled",
        "MPR-R__GradWarp__B1_Correction__N3__Scaled"
    }

    for root_dir in root_dirs:
        print(f"\n Scanning root directory: {root_dir}")
        for subject_id in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject_id)
            if not os.path.isdir(subject_path):
                continue

            # Check if the subject contains any of the desired folders
            subfolders = os.listdir(subject_path)
            has_valid_mpr = any(folder in keep_folders for folder in subfolders)

            if not has_valid_mpr:
                print(f" Deleting subject (no valid MPR): {subject_path}")
                shutil.rmtree(subject_path)
            else:
                print(f" Keeping subject: {subject_path}")

# Example usage:
clean_subjects_with_mpr([
    r"C:\ADNI\AD",
    r"C:\ADNI\CN"
])


 Scanning root directory: C:\ADNI\AD


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\ADNI\\AD'

In [ ]:
"""
now for the pMCI , sMCI groups we will keep the MPR-R/MPR__GradWarp__B1_Correction__N3__Scaled
and the  MPR__GradWarp__N3__Scaled we be saved in a new subfolder to be bias corrected later

"""

import os
import shutil

def process_subjects_for_mpr(root_dirs):
    keep_folders = {
        "MPR__GradWarp__B1_Correction__N3__Scaled",
        "MPR-R__GradWarp__B1_Correction__N3__Scaled"
    }
    preprocess_folder = "NeedsPreprocessing"
    n3_folder = "MPR__GradWarp__N3__Scaled"

    for root_dir in root_dirs:
        print(f"\n Processing root: {root_dir}")
        for subject_id in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject_id)
            if not os.path.isdir(subject_path):
                continue

            subfolders = os.listdir(subject_path)
            has_b1 = any(folder in keep_folders for folder in subfolders)
            has_n3 = n3_folder in subfolders

            if has_b1:
                print(f" Keeping subject with B1: {subject_path}")
                continue

            elif has_n3:
                print(f" Subject with N3 only: {subject_path}")
                # Create NeedsPreprocessing folder if not exists
                preprocess_path = os.path.join(subject_path, preprocess_folder)
                os.makedirs(preprocess_path, exist_ok=True)

                old_n3_path = os.path.join(subject_path, n3_folder)
                new_n3_path = os.path.join(preprocess_path, n3_folder)

                # Move N3 folder to NeedsPreprocessing
                shutil.move(old_n3_path, new_n3_path)
                print(f" Moved {n3_folder} to {preprocess_folder}")

                # Delete any other irrelevant folders
                for folder in subfolders:
                    if folder != preprocess_folder and folder != n3_folder:
                        folder_path = os.path.join(subject_path, folder)
                        if os.path.isdir(folder_path):
                            print(f"🗑️ Deleting other folder: {folder_path}")
                            shutil.rmtree(folder_path)

            else:
                print(f" No valid MPR found. Deleting subject: {subject_path}")
                shutil.rmtree(subject_path)

# Run it:
process_subjects_for_mpr([
    r"C:\ADNI\pMCI",
    r"C:\ADNI\sMCI"
])







 Processing root: C:\ADNI\pMCI


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\ADNI\\pMCI'

In [ ]:
"""
as we want to train and deploy a longitudinal deep learning model its important
to orginaze the data so it would be consistent with the model structure

so we turn such .nii file path from
"ADNI\AD\002_S_0619\MPR-R__GradWarp__N3__Scaled\2007-06-22_07_06_17.0\I67871\ADNI_002_S_0619_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816100717385_S33969_I67871.nii"
 into this
"ADNI_Organized\AD\002_S_0619\timepoint_1\ADNI_002_S_0619_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816100717385_S33969_I67871.nii"

"""

import os
import shutil
from pathlib import Path
from datetime import datetime

# Root of original and destination folders
input_groups = ["pMCI", "sMCI","AD", "CN","NeedsBiasPreprocessingPmci","NeedsBiasPreprocessingSmci"]
source_root = r"C:\ADNI"
destination_root = r"C:\ADNI_Sorted"

for group in input_groups:
    group_path = os.path.join(source_root, group)
    subjects = [s for s in os.listdir(group_path) if os.path.isdir(os.path.join(group_path, s))]

    for subject in subjects:
        subject_path = os.path.join(group_path, subject)

        # Look for scan types like MPR__ or MPR-R__
        for scan_type in os.listdir(subject_path):
            scan_path = os.path.join(subject_path, scan_type)
            if not os.path.isdir(scan_path):
                continue

            # Collect timepoint folders and sort by datetime
            time_folders = []
            for time_folder in os.listdir(scan_path):
                try:
                    time_obj = datetime.strptime(time_folder, "%Y-%m-%d_%H_%M_%S.0")
                    full_path = os.path.join(scan_path, time_folder)
                    time_folders.append((time_obj, full_path))
                except ValueError:
                    continue  # Skip malformed time folders

            # Sort by date
            time_folders.sort()

            # Copy to new organized folder
            for idx, (dt, full_path) in enumerate(time_folders, start=1):
                new_folder = os.path.join(destination_root, group, subject, f"timepoint_{idx}")
                os.makedirs(new_folder, exist_ok=True)

                for root, _, files in os.walk(full_path):
                    for file in files:
                        src = os.path.join(root, file)
                        dst = os.path.join(new_folder, file)
                        shutil.copy2(src, dst)

                print(f" Copied: {subject}, {scan_type}, timepoint_{idx}")

print(" All subjects organized by acquisition time.")
